In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from model import *
from data import prepare_data_vocab, live_feat
from torch.nn.utils.rnn import unpack_sequence
from torch_geometric.data import Data


import torch.optim as optim
from torch_geometric.data import Data

[['store', 'double', 'TMP_VAR(INFUNCTION_VAR)', 'TMP_VAR', ';', 'load', 'TMP_VAR', 'double', 'TMP_VAR(INFUNCTION_VAR)', ';', 'assignment', 'TMP_POINTER_MEMBER', 'TMP_VAR', ';', 'assignment', 'TMP_VAR', 'TMP_POINTER_MEMBER', ';', 'math_op', 'TMP_VAR', 'TMP_NUMBER', '/', 'TMP_VAR', ';', 'function_call', 'INFUNCTION_VAR', 'TMP_FUNCTION_ARG_1', 'TMP_FUNCTION_ARG_2', ';', 'function_call', 'INFUNCTION_VAR', 'TMP_FUNCTION_ARG_1', 'TMP_FUNCTION_ARG_2', 'TMP_FUNCTION_ARG_3', 'TMP_FUNCTION_ARG_4', ';', 'math_op', 'TMP_VAR', 'TMP_VAR', '+', 'TMP_NUMBER', ';', 'assignment', 'INFUNCTION_VAR', 'TMP_VAR', ';', 'phi', ';']]


In [5]:
word_to_idx, idx_to_word, data, data_emb = prepare_data_vocab("data", func=live_feat, function_num=0)

In [6]:
hidden_dim = 13
if torch.cuda.is_available():
    device = torch.device("cuda")  # Or "cuda:0" for the first GPU
else:
    device = torch.device("cpu")
idx = torch.tensor(list(word_to_idx.values())).to(device)

ae = AE_rnn(device, idx, 7, 13, 3)#.to(device)
ae.to(device)

criterion = nn.CrossEntropyLoss()
data_emb_device = [(edge_index.to(device), [bb.to(device) for bb in node_embs ]) for (edge_index, node_embs) in data_emb] 
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb_device:
        j += 1
        edge_index, node_embs = graph
        for bb in node_embs:
            #bb_device = bb.to(device)
            out, embs = ae(bb)
            #print(a.shape)
            #print(out)
            #total_loss += unsupervised_loss.item()
            #loss = criterion(out, embs)
            loss = criterion(out, bb)
            #loss = criterion(output, initial_node_embeddings[j])
            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')

[['store', 'double', 'TMP_VAR(INFUNCTION_VAR)', 'TMP_VAR', ';', 'load', 'TMP_VAR', 'double', 'TMP_VAR(INFUNCTION_VAR)', ';', 'assignment', 'TMP_POINTER_MEMBER', 'TMP_VAR', ';', 'assignment', 'TMP_VAR', 'TMP_POINTER_MEMBER', ';', 'math_op', 'TMP_VAR', 'TMP_NUMBER', '/', 'TMP_VAR', ';', 'function_call', 'INFUNCTION_VAR', 'TMP_FUNCTION_ARG_1', 'TMP_FUNCTION_ARG_2', ';', 'function_call', 'INFUNCTION_VAR', 'TMP_FUNCTION_ARG_1', 'TMP_FUNCTION_ARG_2', 'TMP_FUNCTION_ARG_3', 'TMP_FUNCTION_ARG_4', ';', 'math_op', 'TMP_VAR', 'TMP_VAR', '+', 'TMP_NUMBER', ';', 'assignment', 'INFUNCTION_VAR', 'TMP_VAR', ';', 'phi', ';']]
For 578 functions: Epoch 1/1000, Average Loss: 9.04676


KeyboardInterrupt: 

In [ ]:
hidden_dim = 13
if torch.cuda.is_available():
    device = torch.device("cuda")  # Or "cuda:0" for the first GPU
else:
    device = torch.device("cpu")
idx = torch.tensor(list(word_to_idx.values())).to(device)

ae = AE_rnn(device, idx, 7, 13, 3)#.to(device)
ae.to(device)

criterion = nn.CrossEntropyLoss()
data_emb_device = [(edge_index.to(device), [bb.to(device) for bb in node_embs ]) for (edge_index, node_embs) in data_emb] 
optimizer = optim.Adam(ae.parameters(), lr=0.01)
epoch_num = 1000
for epoch in range(epoch_num):
    total_loss = 0
    total_items = 0
    j = -1
    for graph in data_emb_device:
        j += 1
        edge_index, node_embs = graph
        for bb in node_embs:
            #bb_device = bb.to(device)
            out, embs = ae(bb)
            #print(a.shape)
            #print(out)
            #total_loss += unsupervised_loss.item()
            #loss = criterion(out, embs)
            loss = criterion(out, bb)
            #loss = criterion(output, initial_node_embeddings[j])
            # Backward pass and optimization
            total_loss  += loss.item()
            total_items += bb.shape[0]

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
    # Print or log the average loss for monitoring
    if epoch%10==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\r')
    if epoch%100==0 or epoch==epoch_num-1:
        avg_loss = total_loss / len(data_emb)
        print(f'For {j+1} functions: Epoch {epoch+1}/{epoch_num}, Average Loss: {avg_loss:.5f}', end='\n')